In [1]:
import pandas as pd

from parse import parse
from datetime import datetime

from random import randint, uniform

import os
import gpxpy

In [2]:
df = pd.read_xml('export.xml')

In [47]:
parsed = df.drop(columns=['HKCharacteristicTypeIdentifierDateOfBirth', 'HKCharacteristicTypeIdentifierBiologicalSex',
                'HKCharacteristicTypeIdentifierBloodType', 'HKCharacteristicTypeIdentifierFitzpatrickSkinType',
                'HKCharacteristicTypeIdentifierCardioFitnessMedicationsUse', 'HeartRateVariabilityMetadataList', 
                'MetadataEntry', 'totalDistanceUnit', 'durationUnit', 'totalEnergyBurnedUnit', 'WorkoutEvent', 
                'WorkoutRoute', 'activeEnergyBurnedGoal', 'activeEnergyBurnedUnit', 'appleMoveTime', 
                'appleMoveTimeGoal', 'appleExerciseTimeGoal', 'appleStandHoursGoal'])

In [48]:
# duration in minutes
# distance in  miles
# energy burned in calories
parsed.columns

Index(['value', 'type', 'sourceName', 'sourceVersion', 'unit', 'creationDate',
       'startDate', 'endDate', 'device', 'workoutActivityType', 'duration',
       'totalDistance', 'totalEnergyBurned', 'dateComponents',
       'activeEnergyBurned', 'appleExerciseTime', 'appleStandHours'],
      dtype='object')

In [62]:
parsed['type'].unique()

array([None, 'HKQuantityTypeIdentifierHeight',
       'HKQuantityTypeIdentifierBodyMass',
       'HKQuantityTypeIdentifierHeartRate',
       'HKQuantityTypeIdentifierRespiratoryRate',
       'HKQuantityTypeIdentifierStepCount',
       'HKQuantityTypeIdentifierDistanceWalkingRunning',
       'HKQuantityTypeIdentifierBasalEnergyBurned',
       'HKQuantityTypeIdentifierActiveEnergyBurned',
       'HKQuantityTypeIdentifierFlightsClimbed',
       'HKQuantityTypeIdentifierAppleExerciseTime',
       'HKQuantityTypeIdentifierDistanceCycling',
       'HKQuantityTypeIdentifierDistanceSwimming',
       'HKQuantityTypeIdentifierSwimmingStrokeCount',
       'HKQuantityTypeIdentifierRestingHeartRate',
       'HKQuantityTypeIdentifierVO2Max',
       'HKQuantityTypeIdentifierWalkingHeartRateAverage',
       'HKQuantityTypeIdentifierHeadphoneAudioExposure',
       'HKQuantityTypeIdentifierWalkingDoubleSupportPercentage',
       'HKQuantityTypeIdentifierSixMinuteWalkTestDistance',
       'HKQuantityType

In [63]:
parsed.head(10)

,value,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,device,workoutActivityType,duration,totalDistance,totalEnergyBurned,dateComponents,activeEnergyBurned,appleExerciseTime,appleStandHours
0,2022-04-17 18:56:18 -0400,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN
2,5.75,HKQuantityTypeIdentifierHeight,Arun’s Apple Watch,7.6.1,ft,2021-09-13 21:01:29 -0400,2021-09-13 21:01:29 -0400,2021-09-13 21:01:29 -0400,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN
3,5.75,HKQuantityTypeIdentifierHeight,Arun’s iPhone,14.6,ft,2021-06-22 19:35:53 -0400,2021-06-22 19:35:53 -0400,2021-06-22 19:35:53 -0400,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN
4,155,HKQuantityTypeIdentifierBodyMass,Arun’s Apple Watch,7.6.1,lb,2021-09-13 21:01:29 -0400,2021-09-13 21:01:29 -0400,2021-09-13 21:01:29 -0400,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN
5,150,HKQuantityTypeIdentifierBodyMass,Arun’s iPhone,14.6,lb,2021-06-22 19:35:53 -0400,2021-06-22 19:35:53 -0400,2021-06-22 19:35:53 -0400,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN
6,77.5777,HKQuantityTypeIdentifierHeartRate,Arun’s Apple Watch,7.6.1,count/min,2021-09-13 21:04:54 -0400,2021-09-13 21:03:50 -0400,2021-09-13 21:03:50 -0400,"<<HKDevice: 0x282c66170>, name:Apple Watch, ma...",None,NaN,NaN,NaN,None,NaN,NaN,NaN
7,99,HKQuantityTypeIdentifierHeartRate,Arun’s Apple Watch,7.6.1,count/min,2021-08-24 18:33:06 -0400,2021-08-24 18:12:18 -0400,2021-08-24 18:12:18 -0400,"<<HKDevice: 0x282c677f0>, name:Apple Watch, ma...",None,NaN,NaN,NaN,None,NaN,NaN,NaN
8,127,HKQuantityTypeIdentifierHeartRate,Arun’s Apple Watch,7.6.1,count/min,2021-08-24 18:42:41 -0400,2021-08-24 18:33:01 -0400,2021-08-24 18:33:01 -0400,"<<HKDevice: 0x282c677f0>, name:Apple Watch, ma...",None,NaN,NaN,NaN,None,NaN,NaN,NaN
9,112,HKQuantityTypeIdentifierHeartRate,Arun’s Apple Watch,7.6.1,count/min,2021-08-24 18:43:48 -0400,2021-08-24 18:42:37 -0400,2021-08-24 18:42:37 -0400,"<<HKDevice: 0x282c677f0>, name:Apple Watch, ma...",None,NaN,NaN,NaN,None,NaN,NaN,NaN


In [64]:
def fmt_hr_date(date):
    return str(datetime.strptime(date, '%Y-%m-%d %H:%M:%S %z').replace(second=0, minute=0, hour=0, tzinfo=None))[:-9]

resting_hr = parsed[parsed['type'] == 'HKQuantityTypeIdentifierRestingHeartRate']
resting_hr['dateComponents'] = resting_hr['creationDate'].apply(fmt_hr_date)
resting_hr = resting_hr[['value', 'dateComponents']]
resting_hr.columns = ['hr', 'dateComponents']
resting_hr

/var/folders/bp/07r6ly0j6gzd_54brsn_ts980000gn/T/ipykernel_3050/895709169.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resting_hr['dateComponents'] = resting_hr['creationDate'].apply(fmt_hr_date)


,hr,dateComponents
723169,86,2021-08-24
723170,70,2021-08-25
723171,66,2021-08-26
723172,69,2021-08-27
723173,80,2021-08-29
...,...,...
723465,70,2022-04-13
723466,70,2022-04-14
723467,67,2022-04-15
723468,61,2022-04-16


In [65]:
import numpy as np

def fmt_hr_date(date):
    return str(datetime.strptime(date, '%Y-%m-%d %H:%M:%S %z').replace(second=0, minute=0, hour=0, tzinfo=None))[:-9]

distance = parsed[(parsed['type'] == 'HKQuantityTypeIdentifierDistanceWalkingRunning') & (parsed['sourceName'] != 'Arun’s iPhone')]
distance['dateComponents'] = distance['creationDate'].apply(fmt_hr_date)
distance['value'] = distance['value'].apply(lambda x: float(x))
distance = distance.groupby(['dateComponents'], as_index=False).agg({'value': 'sum'})
distance.columns = ['dateComponents', 'distance']
distance

/var/folders/bp/07r6ly0j6gzd_54brsn_ts980000gn/T/ipykernel_3050/2998687855.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distance['dateComponents'] = distance['creationDate'].apply(fmt_hr_date)
/var/folders/bp/07r6ly0j6gzd_54brsn_ts980000gn/T/ipykernel_3050/2998687855.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distance['value'] = distance['value'].apply(lambda x: float(x))


,dateComponents,distance
0,2021-04-23,0.034632
1,2021-06-22,0.181932
2,2021-06-23,1.727688
3,2021-06-24,2.182513
4,2021-06-25,3.419816
...,...,...
296,2022-04-13,1.504423
297,2022-04-14,3.886069
298,2022-04-15,4.789115
299,2022-04-16,0.745918


In [66]:
hr_dist = resting_hr.merge(distance, on='dateComponents')
hr_dist

,hr,dateComponents,distance
0,86,2021-08-24,2.600717
1,70,2021-08-25,1.777992
2,66,2021-08-26,2.109896
3,69,2021-08-27,2.006553
4,80,2021-08-29,2.603741
...,...,...,...
296,70,2022-04-13,1.504423
297,70,2022-04-14,3.886069
298,67,2022-04-15,4.789115
299,61,2022-04-16,0.745918


In [67]:
goals = parsed[parsed['dateComponents'].notnull()]
goals = goals[['dateComponents', 'activeEnergyBurned', 'appleExerciseTime', 'appleStandHours']]
goals = goals.merge(hr_dist, on='dateComponents', how='left')
goals

,dateComponents,activeEnergyBurned,appleExerciseTime,appleStandHours,hr,distance
0,2021-04-22,0.000,0.0,0.0,NaN,NaN
1,2021-04-23,6.014,0.0,0.0,NaN,NaN
2,2021-06-21,0.000,0.0,0.0,NaN,NaN
3,2021-06-22,44.201,1.0,3.0,77,0.181932
4,2021-06-23,597.345,54.0,13.0,69,1.727688
...,...,...,...,...,...,...
338,2022-04-13,297.248,11.0,9.0,70,1.504423
339,2022-04-14,714.792,55.0,17.0,70,3.886069
340,2022-04-15,572.192,34.0,10.0,67,4.789115
341,2022-04-16,220.634,4.0,6.0,61,0.745918


In [68]:
def fmt_wkt_date(date):
    return datetime.strptime(date, '%Y-%m-%d %H:%M:%S %z').replace(second=0, tzinfo=None)

workouts = parsed[parsed['workoutActivityType'].notnull()]
workouts = workouts[['creationDate', 'startDate', 'workoutActivityType', 'duration', 'totalDistance', 'totalEnergyBurned']]
workouts['fmtDate'] = workouts['creationDate'].apply(fmt_wkt_date)
workouts.to_csv('workouts.csv')
workouts

,creationDate,startDate,workoutActivityType,duration,totalDistance,totalEnergyBurned,fmtDate
789525,2021-06-23 11:10:53 -0400,2021-06-23 10:45:20 -0400,HKWorkoutActivityTypeFunctionalStrengthTraining,25.535174,0.000000,156.172000,2021-06-23 11:10:00
789526,2021-06-23 11:26:04 -0400,2021-06-23 11:11:06 -0400,HKWorkoutActivityTypeRunning,14.962198,1.269588,160.351839,2021-06-23 11:26:00
789527,2021-06-24 10:58:15 -0400,2021-06-24 10:21:19 -0400,HKWorkoutActivityTypeTraditionalStrengthTraining,36.924798,0.000000,321.019050,2021-06-24 10:58:00
789528,2021-06-24 11:13:44 -0400,2021-06-24 10:59:12 -0400,HKWorkoutActivityTypeRunning,13.240177,1.272011,145.764093,2021-06-24 11:13:00
789529,2021-06-25 12:03:06 -0400,2021-06-25 11:36:34 -0400,HKWorkoutActivityTypeFunctionalStrengthTraining,26.534099,0.000000,136.759644,2021-06-25 12:03:00
...,...,...,...,...,...,...,...
789655,2022-04-08 13:51:44 -0400,2022-04-08 13:27:11 -0400,HKWorkoutActivityTypeRunning,24.532083,2.126172,241.619691,2022-04-08 13:51:00
789656,2022-04-09 15:44:25 -0400,2022-04-09 14:57:26 -0400,HKWorkoutActivityTypeWalking,41.481165,1.980508,158.121756,2022-04-09 15:44:00
789657,2022-04-14 07:49:59 -0400,2022-04-14 07:30:20 -0400,HKWorkoutActivityTypeTraditionalStrengthTraining,19.638143,0.000000,157.348126,2022-04-14 07:49:00
789658,2022-04-14 22:41:20 -0400,2022-04-14 22:23:50 -0400,HKWorkoutActivityTypeWalking,17.487966,0.715353,52.198115,2022-04-14 22:41:00


In [69]:
points = []
path = "/Users/arundhingra/Downloads/apple_health_export/workout-routes"
dir_list = os.listdir(path)

def fmt_route_date(route_date):
    res = parse('{:d}-{:d}-{:d} {:d}:{:d}{:l}', route_date)
    route_date = "{}-{:02d}-{:02d} {:02d}:{:02d}{}".format(res[0], res[1], 
                                                     res[2], res[3], 
                                                     res[4], res[5])
    return datetime.strptime(route_date, '%Y-%m-%d %I:%M%p')

for path in dir_list:
    gpx = gpxpy.parse(open("workout-routes/{}".format(path)))
    first_point = gpx.tracks[0].segments[0].points[0]
    date = gpx.tracks[0].name[6:]
    points.append((fmt_route_date(date), first_point.latitude, first_point.longitude))

    
points

[(datetime.datetime(2021, 8, 1, 17, 1), 38.994252, -76.934492),
 (datetime.datetime(2022, 2, 11, 21, 4), 38.990239, -76.935258),
 (datetime.datetime(2022, 2, 12, 19, 0), 38.990501, -76.934526),
 (datetime.datetime(2021, 6, 27, 0, 15), 38.981211, -76.937943),
 (datetime.datetime(2021, 6, 29, 17, 28), 38.714705, -75.074474),
 (datetime.datetime(2022, 4, 9, 15, 44), 38.9903, -76.93442),
 (datetime.datetime(2022, 2, 19, 14, 27), 38.99068, -76.934256),
 (datetime.datetime(2022, 2, 16, 19, 54), 38.990727, -76.934219),
 (datetime.datetime(2021, 7, 30, 21, 22), 38.990539, -76.934216),
 (datetime.datetime(2022, 2, 13, 14, 39), 38.990788, -76.934272),
 (datetime.datetime(2022, 4, 14, 22, 41), 38.989509, -76.93473),
 (datetime.datetime(2022, 2, 12, 14, 51), 38.990766, -76.934388),
 (datetime.datetime(2021, 7, 30, 23, 30), 38.980715, -76.938359),
 (datetime.datetime(2021, 6, 29, 18, 15), 38.714801, -75.074307),
 (datetime.datetime(2021, 8, 13, 18, 57), 38.994287, -76.93463),
 (datetime.datetime(20

In [70]:
from datetime import timedelta
dates = list(map(lambda x: x[0], points))
matched = []
margin = timedelta(hours = 2)
new_workout_dates = []
for i in workouts['fmtDate']:
    found = [x for x in dates if x - margin < i < x + margin]
    if found:
        new_workout_dates.append(pd.Timestamp(found[0]))
        dates.remove(found[0])
    else:
        new_workout_dates.append(i)

workouts['fmtDate'] = new_workout_dates

In [71]:
workouts['onlyDate'] = workouts['fmtDate'].apply(lambda x: x.replace(second=0, minute=0, hour=0))

In [72]:
varsity = (38.99089726384877, -76.9343323725462)
deep_creek = (39.514183, -79.309469)
rehoboth = (38.733435, -75.078909)
echo = (38.919791, -76.972322)
room = (38.99197562859428, -76.93441835548659)
outside_varsity = (38.99295439860751, -76.93546173743849)
home = (38.922336476725015, -77.37557836807055)

together = []
loc = [
    varsity,varsity,varsity,varsity,varsity,varsity,
    varsity,varsity,rehoboth,varsity,varsity,varsity,
    varsity,varsity,varsity,varsity,varsity,varsity,
    varsity,varsity,varsity,varsity,varsity,varsity,
    varsity,varsity,varsity,varsity,varsity,varsity,
    varsity,deep_creek,deep_creek,deep_creek,varsity,varsity,
    varsity,varsity,varsity,varsity,room,varsity,
    varsity,echo,echo,room,varsity,varsity,
    varsity,varsity,varsity,varsity,varsity,varsity,
    varsity,room,varsity,varsity,varsity,varsity,
    varsity,varsity,varsity,varsity,varsity,varsity,
    varsity,varsity,home,varsity,varsity,varsity,
    home,home,home,home,home,home,
    varsity,home,home,home,home,home,
    home,home,home,home,varsity,varsity,
    varsity,varsity,room,room,outside_varsity,room,
    room,varsity,room,room,room,varsity,
    varsity,room,varsity,varsity,varsity,room,
    outside_varsity,varsity,room
]

In [73]:
tracked = pd.DataFrame(points, columns=['fmtDate', 'latitude', 'longitude'])
joined = workouts.merge(tracked, on='fmtDate', how='left')
joined

,creationDate,startDate,workoutActivityType,duration,totalDistance,totalEnergyBurned,fmtDate,onlyDate,latitude,longitude
0,2021-06-23 11:10:53 -0400,2021-06-23 10:45:20 -0400,HKWorkoutActivityTypeFunctionalStrengthTraining,25.535174,0.000000,156.172000,2021-06-23 11:10:00,2021-06-23,NaN,NaN
1,2021-06-23 11:26:04 -0400,2021-06-23 11:11:06 -0400,HKWorkoutActivityTypeRunning,14.962198,1.269588,160.351839,2021-06-23 11:26:00,2021-06-23,NaN,NaN
2,2021-06-24 10:58:15 -0400,2021-06-24 10:21:19 -0400,HKWorkoutActivityTypeTraditionalStrengthTraining,36.924798,0.000000,321.019050,2021-06-24 10:58:00,2021-06-24,NaN,NaN
3,2021-06-24 11:13:44 -0400,2021-06-24 10:59:12 -0400,HKWorkoutActivityTypeRunning,13.240177,1.272011,145.764093,2021-06-24 11:13:00,2021-06-24,NaN,NaN
4,2021-06-25 12:03:06 -0400,2021-06-25 11:36:34 -0400,HKWorkoutActivityTypeFunctionalStrengthTraining,26.534099,0.000000,136.759644,2021-06-25 12:03:00,2021-06-25,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
130,2022-04-08 13:51:44 -0400,2022-04-08 13:27:11 -0400,HKWorkoutActivityTypeRunning,24.532083,2.126172,241.619691,2022-04-08 13:51:00,2022-04-08,NaN,NaN
131,2022-04-09 15:44:25 -0400,2022-04-09 14:57:26 -0400,HKWorkoutActivityTypeWalking,41.481165,1.980508,158.121756,2022-04-09 15:44:00,2022-04-09,38.990300,-76.93442
132,2022-04-14 07:49:59 -0400,2022-04-14 07:30:20 -0400,HKWorkoutActivityTypeTraditionalStrengthTraining,19.638143,0.000000,157.348126,2022-04-14 07:49:00,2022-04-14,NaN,NaN
133,2022-04-14 22:41:20 -0400,2022-04-14 22:23:50 -0400,HKWorkoutActivityTypeWalking,17.487966,0.715353,52.198115,2022-04-14 22:41:00,2022-04-14,38.989509,-76.93473


In [74]:
from math import isnan

longs = joined['longitude']
lats = joined['latitude']
date = joined['fmtDate']

i = 0
li = 0
while i < len(longs):
    if isnan(longs[i]):
        lats[i] = loc[li][0]
        longs[i] = loc[li][1]
        li += 1
    i += 1
        
joined['longitude'] = longs
joined['latitude'] = lats
joined['workoutActivityType'] = joined['workoutActivityType'].apply(lambda x: x.replace('HKWorkoutActivityType', ''))

/var/folders/bp/07r6ly0j6gzd_54brsn_ts980000gn/T/ipykernel_3050/1089136307.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lats[i] = loc[li][0]
/var/folders/bp/07r6ly0j6gzd_54brsn_ts980000gn/T/ipykernel_3050/1089136307.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  longs[i] = loc[li][1]


In [75]:
import random

def jitter(coords):
    new_lat = uniform(0,.01)
    new_long = uniform(0,.01)
        
    if (random.randint(1, 50) % 2 == 0):
        new_lat += coords[0]
    else:
        new_lat = coords[0] - new_lat
        
    if (random.randint(1, 50) % 2 == 0):
        new_long += coords[1]
    else:
        new_long = coords[1] - new_long
        
    return (new_lat, new_long)

zipped = zip(joined['latitude'], joined['longitude'])
jittered = list(map(jitter, zipped))
joined['latitude'] = list(map(lambda x: x[0], jittered))
joined['longitude'] = list(map(lambda x: x[1], jittered))
joined

,creationDate,startDate,workoutActivityType,duration,totalDistance,totalEnergyBurned,fmtDate,onlyDate,latitude,longitude
0,2021-06-23 11:10:53 -0400,2021-06-23 10:45:20 -0400,FunctionalStrengthTraining,25.535174,0.000000,156.172000,2021-06-23 11:10:00,2021-06-23,38.991265,-76.935500
1,2021-06-23 11:26:04 -0400,2021-06-23 11:11:06 -0400,Running,14.962198,1.269588,160.351839,2021-06-23 11:26:00,2021-06-23,38.995282,-76.942194
2,2021-06-24 10:58:15 -0400,2021-06-24 10:21:19 -0400,TraditionalStrengthTraining,36.924798,0.000000,321.019050,2021-06-24 10:58:00,2021-06-24,38.999788,-76.941503
3,2021-06-24 11:13:44 -0400,2021-06-24 10:59:12 -0400,Running,13.240177,1.272011,145.764093,2021-06-24 11:13:00,2021-06-24,38.989353,-76.939716
4,2021-06-25 12:03:06 -0400,2021-06-25 11:36:34 -0400,FunctionalStrengthTraining,26.534099,0.000000,136.759644,2021-06-25 12:03:00,2021-06-25,38.987979,-76.930579
...,...,...,...,...,...,...,...,...,...,...
130,2022-04-08 13:51:44 -0400,2022-04-08 13:27:11 -0400,Running,24.532083,2.126172,241.619691,2022-04-08 13:51:00,2022-04-08,38.998019,-76.933432
131,2022-04-09 15:44:25 -0400,2022-04-09 14:57:26 -0400,Walking,41.481165,1.980508,158.121756,2022-04-09 15:44:00,2022-04-09,38.983032,-76.926405
132,2022-04-14 07:49:59 -0400,2022-04-14 07:30:20 -0400,TraditionalStrengthTraining,19.638143,0.000000,157.348126,2022-04-14 07:49:00,2022-04-14,38.994451,-76.936152
133,2022-04-14 22:41:20 -0400,2022-04-14 22:23:50 -0400,Walking,17.487966,0.715353,52.198115,2022-04-14 22:41:00,2022-04-14,38.995789,-76.940305


In [76]:
joined.to_csv('workouts.csv')
goals.to_csv('goals.csv')

In [77]:
len(joined[joined['workoutActivityType'] == 'Running'])

33